In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 타입 맞춤

In [2]:
data = pd.read_csv('Final.csv')

In [3]:
data_list = data.values.tolist()

In [4]:
item_list = []

for i in data_list:
    item_list.append(list(map(str,i)))

In [5]:
data = data.astype(float)

In [6]:
data = data.applymap(str)

# Word2Vec

In [7]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

In [8]:
path = get_tmpfile('word2vec.model')

In [9]:
model = Word2Vec(item_list, size=1 ,window=5, min_count=1,workers=5)
model.save('word2vec.model')

In [10]:
model = Word2Vec.load('word2vec.model')

In [11]:
word_vectors = model.wv  
vocabs = word_vectors.vocab.keys() #vocabs has keys that is item number

#word_vectors_list has values that is vector value from item number
word_vectors_list = [word_vectors[v] for v in vocabs] 

In [133]:
for i in range(len(data)):            #word2vec value를 data로 넣음
    for j in range(len(data.iloc[i])):
        key=str(data.iloc[i,j])
        data.iloc[i,j] = word_vectors[key].tolist()

In [134]:
data = data.apply(pd.to_numeric)

TypeError: Invalid object type at position 0

# Word2Vec값과 아이템 맞춤

In [ ]:
num_vec = {}

In [ ]:
item_id = word_vectors.vocab

In [ ]:
item_num = []
for i in item_id.keys():
    item_num.append(i)

In [ ]:
item_vec = []
for i in vocabs:
    item_vec.append(word_vectors[i][0])

In [ ]:
num_vec = num_vec.fromkeys(item_vec)

In [ ]:
j = 0
for i in num_vec.keys():
    num_vec[i] = item_num[j]
    j = j+1

In [ ]:
num_vec

# XGBoost

In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
from heapq import nsmallest
import time

In [ ]:
X_data = data.drop('13',axis=1)
y_data = data['13']

In [ ]:
dtrain = xgb.DMatrix(data=X_data, label=y_data)
params = {
    "Objective": 'gbtree',
    "eval_metric": 'mae',
    "eta" : 0.1 #learning late
}
cv = xgb.cv(params=params, dtrain=dtrain, num_boost_round=100, nfold=5,
            seed=6, early_stopping_rounds= 10)
cv

In [ ]:
import matplotlib.pyplot as plt
plt.plot(cv['train-mae-mean'])
plt.xlabel("")
plt.ylabel("")
plt.xlim(1,100)
plt.ylim(0,1)
plt.rcParams["figure.figsize"] = 6,4
plt.show()

In [ ]:
start = time.time()
seed = 6
test_size = 0.2
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data,test_size=test_size, random_state=seed)
model = xgb.XGBClassifier(n_estimators=500,max_depth=8,learning_rate=0.1,subsample=0.5)
model.fit(X_train,y_train)
end = time.time()
print(end-start)

In [ ]:
result = pd.DataFrame()

In [ ]:
prediction = model.predict(X_test)

In [ ]:
result['predicted'] = prediction
result['Y_test'] = y_test.tolist()

In [ ]:
vector_list = []

for i in range(len(result)):
    vector_list.append(nsmallest(5, word_vectors_list, key=lambda x: abs(x-result['predicted'].iloc[i])))
    
result['Nearby5'] = pd.Series(vector_list)
    
for i in range(len(result)):
    result['Nearby5'].iloc[i] = vector_list[i]

T_F = []
for i in range(len(result)):
    if result['Y_test'].iloc[i] in result['Nearby5'].iloc[i]:
        T_F.append('True')
    else:
        T_F.append('False')
    
result['T_F'] = T_F

In [ ]:
total_list = []
for i in range(len(result)):
    vector_list = []
    for j in range(5):
        vector_list.append(nsmallest(5, word_vectors_list, key=lambda x: abs(x-result['predicted'].iloc[i]))[j][0])
    total_list.append(vector_list)

In [ ]:
real_itemnum = []
for i in range(len(total_list)):
    real_item = []
    for j in range(5):
        real_item.append(num_vec[total_list[i][j]])
    real_itemnum.append(real_item)

In [ ]:
result['Item5'] = real_itemnum

In [ ]:
result

In [ ]:
result['Item5'][0]

In [ ]:
data